In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
date = "2025-11"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [15]:
end_use = "naics?get=CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_imports"] = df["CON_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum","CTY_CODE":"first"})

country_list = list(top_products.sort_values(by = "total_imports", ascending = False).CTY_CODE)[0:31]


['TOTAL FOR ALL COUNTRIES','NAFTA','EUROPEAN UNION']

<Response [200]>


['TOTAL FOR ALL COUNTRIES', 'NAFTA', 'EUROPEAN UNION']

In [16]:
df.tail()

,CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL,time,total_imports
39049,13896714,7940,ZAMBIA,DET,2025-11,13896714.0
39050,12624834,7950,ESWATINI,DET,2025-11,12624834.0
39051,15520151,7960,ZIMBABWE,DET,2025-11,15520151.0
39052,2368430,7970,MALAWI,DET,2025-11,2368430.0
39053,8270767,7990,LESOTHO,DET,2025-11,8270767.0


In [17]:
df.time.unique()[-1]

'2025-11'

In [18]:
country_list[0] = ""

In [19]:
country_list.extend(["0003", "0020"])

In [21]:
end_use = "naics?get=CTY_NAME,CON_VAL_MO,CAL_DUT_MO,NAICS,NAICS_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

surl  = surl + my_key + "&time=" + "from+2013-01" + "&COMM_LVL=NA3" 

for xxx in country_list:
    
    out_file = "..\\data"+ "\\imports\\" + xxx + "naics-data-" + date + ".parquet"
    
    if xxx == "":
        out_file = "..\\data"+ "\\imports\\" + "TOTAL" + "naics-data-" + date + ".parquet"
    
    
    if os.path.exists(out_file):
        
        print("Already have downloaded file")
        
        continue
    
    print(xxx)
    
    url = surl + "&CTY_CODE=" + xxx
    
    if xxx == "":
        url = surl
    
    r = requests.get(url) 
    
    print(r)
    
    foo = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    foo.columns = r.json()[0]

    pq.write_table(pa.Table.from_pandas(foo), out_file)

Already have downloaded file
5700
<Response [200]>
2010
<Response [200]>
1220
<Response [200]>
5880
<Response [200]>
4280
<Response [200]>
5800
<Response [200]>
5520
<Response [200]>
5830
<Response [200]>
4190
<Response [200]>
5330
<Response [200]>
4120
<Response [200]>
4759
<Response [200]>
4279
<Response [200]>
4419
<Response [200]>
5490
<Response [200]>
5570
<Response [200]>
3510
<Response [200]>
5590
<Response [200]>
4210
<Response [200]>
5600
<Response [200]>
5081
<Response [200]>
4231
<Response [200]>
5170
<Response [200]>
4700
<Response [200]>
4621
<Response [200]>
3010
<Response [200]>
6021
<Response [200]>
4330
<Response [200]>
4010
<Response [200]>
3370
<Response [200]>
0003
<Response [200]>
0020
<Response [200]>


In [23]:
foo.NAICS.unique()

array(['321', '331', '327', '333', '336', '990', '930', '980', '910',
       '311', '211', '212', '315', '314', '335', '334', '339', '337',
       '114', '316', '326', '325', '332', '112', '312', '322', '323',
       '324', '113', '111', '313', '115'], dtype=object)

In [25]:
country_list[0] = "TOTAL"

df = pd.DataFrame([])
#this is for all countries...

for xxx in country_list:
    
    file = "..\\data\\imports\\" + xxx + "naics-data-" + date + ".parquet"

    foo = pq.read_table(file).to_pandas()
        
    df = pd.concat([df, foo], ignore_index=True)
            
df.time = pd.to_datetime(df.time, format="%Y-%m")

df["imports"] = df["CON_VAL_MO"].astype(float)

df["duty"] = df["CAL_DUT_MO"].astype(float)

total = df.groupby(["CTY_NAME", "time"]).agg({"imports": "sum", "duty": "sum"})
                   
total.reset_index(inplace = True)
                   
total["NAICS"] = "00"
total["NAICS_SDESC"] = "ALL PRODUCTS"
                   
df = pd.concat([df, total], ignore_index=True)

df["itariff"] = 100*df["duty"]/df["imports"]

df.drop(["CON_VAL_MO", "COMM_LVL", "CTY_CODE", "CAL_DUT_MO"], axis = 1, inplace = True)

df["NAICS_SDESC"] = "NAICS CODE " + df["NAICS"] + ", " + df["NAICS_SDESC"]

df["NAICS_SDESC"] = df["NAICS_SDESC"].replace({"NAICS CODE 00, ALL PRODUCTS": "ALL PRODUCTS"})

df.set_index(["CTY_NAME","NAICS_SDESC","time"],inplace = True)

out_file = "..\\data"+ "\\top30-NAICS-imports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [29]:
foo = pq.read_table("..\\data\\top30-NAICS-imports.parquet").to_pandas()

In [30]:
foo.tail()

NAICS       imports          duty    itariff
CTY_NAME NAICS_SDESC  time                                                   
VIETNAM  ALL PRODUCTS 2025-07-01    00  1.776576e+10  1.640984e+09   9.236779
                      2025-08-01    00  1.723779e+10  1.722283e+09   9.991325
                      2025-09-01    00  1.858137e+10  2.343462e+09  12.611886
                      2025-10-01    00  1.958902e+10  2.412794e+09  12.317073
                      2025-11-01    00  1.794097e+10  2.185444e+09  12.181308

In [ ]:
foo.loc["TOTAL FOR ALL COUNTRIES", "HS CODE 01, LIVE ANIMALS"]

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2013-01-01,01,221313094.0,417.0,0.000188
2013-02-01,01,246092109.0,823.0,0.000334
2013-03-01,01,261244392.0,75.0,0.000029
2013-04-01,01,271133926.0,1389.0,0.000512
2013-05-01,01,209785289.0,50.0,0.000024
...,...,...,...,...
2025-07-01,01,266619048.0,3777405.0,1.416780
2025-08-01,01,215515069.0,3748886.0,1.739501
2025-09-01,01,273013211.0,4814765.0,1.763565


In [32]:
foo.to_csv("..\\data\\top30-NAICS-imports.csv")